In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import time
import datetime

<h2>DeepAR - Kaggle Bike Sharing Demand Dataset with Dynamic Features</h2>
<h4>Prepare Bike Rental Data for DeepAR training</h4>  
<quote>We will store the total rental count, registered rental count, and casual rental count data as a time series.</quote>
<quote>For each of the targets <b>total, registered, casual</b>, following JSON is structured as: Start Time, Array of target values, optional dynamic features and categories</quote>
<quote>Frequency of time series data (for example, hourly, daily, monthly and so forth) is specified using hyperparameter</quote>

<h4>To download original dataset, sign-in and download from this link: https://www.kaggle.com/c/bike-sharing-demand/data</h4>
<br>
<b>Dynamic features used</b>: ['season', 'holiday', 'workingday', 'weather', 'temp',
       'atemp', 'humidity', 'windspeed']<br>
       
Start Time From: ['datetime'] <br>
Target Feature: [<b>'count','registered','casual'</b>]<br>
Frequency: 'Hourly' <br>

Objective: <quote>You are provided hourly rental data spanning two years. For this competition, the training set is comprised of the first 19 days of each month, while the test set is the 20th to the end of the month. You must predict the total count of bikes rented during each hour covered by the test set, using only information available prior to the rental period (Ref: Kaggle.com)</quote>

In [ ]:
dynamic_features = ['season', 'holiday', 'workingday', 'weather', 'temp', 'atemp', 'humidity', 'windspeed']
target_values = ['count','registered','casual']

In [ ]:
# Copy over from biketrain_data_preparation

freq='H' # Timeseries consists Hourly Data and we need to predict hourly rental count

# how far in the future predictions can be made
# 12 days worth of hourly forecast 
prediction_length = 288 

# aws recommends setting context same as prediction length as a starting point. 
# This controls how far in the past the network can see
context_length = 288

In [ ]:
dt_predict_max = pd.Timestamp("2012-12-31 23:00:00", freq=freq) # 2012-12-31 23:00 alt way..pd.datetime(2012,12,31,23,0,0)

dt_dataset_start_time = pd.Timestamp("2011-01-01 00:00:00", freq=freq)
dt_dataset_end_time = pd.Timestamp("2012-12-19 23:00:00", freq=freq)

# use for model training
# Start time is the first row provided by kaggle
# Training TS end time ensures some data is withheld for model testing
# 12 days worth of training data is withheld for testing
dt_train_range = (dt_dataset_start_time,
                  dt_dataset_end_time - datetime.timedelta(hours=12*24) )

# Use entire data for testing
# We can compare predicted values vs actual (i.e. last 12 days is withheld for testing and model hasn't seen that data)
dt_test_range = (dt_dataset_start_time, 
                 dt_dataset_end_time) 

In [ ]:
# Let's see if there are gaps in timesteps
def is_missing_steps(df,start,end,freq='D'):
    dt_range = pd.date_range(start=start,end=end,freq=freq)
    return not dt_range.equals(df[start:end].index)

def get_missing_steps(df,start,end,freq='D'):
    dt_range = pd.date_range(start=start,end=end,freq=freq)
    return dt_range.difference(df[start:end].index)    

# List timeseries with only NaNs
# They can be removed
def timeseries_with_only_nans(df):
    l = []
    for col in df.columns:
        if pd.isna(df[col].min()):
            #print (col)
            l.append(col)
    return l

In [ ]:
df = pd.read_csv('train.csv', parse_dates=['datetime'],index_col=0)
df_test = pd.read_csv('test.csv', parse_dates=['datetime'],index_col=0)

In [ ]:
# Dynamic Features for future bike count that needs to be predicted is available in two separate files
# train.csv = contains dynamic features for model training and evaluation (i.e. upto 19th of each month)
# test.csv = contains dynamic features for predictions that we need to submit to kaggle. (i.e from 20th to end of each month)
# For model training with dynamic features, we need to provide dynamic features for missing time steps

In [ ]:
df['2011-01']['temp'].plot(title='Temperature upto 19th day - train.csv')

In [ ]:
df_test['2011-01']['temp'].plot(title='Temperature from 20th day to end of month - test.csv')

In [ ]:
# Let's merge train.csv and test.csv files to a single dataframe
# This will allow dynamic features for missing time steps to populated from test.csv file
# NOTE: There are some missing time steps in test.csv as well...we need to handle it

In [ ]:
df_test['2011-01'].index.min(),df_test['2011-01'].index.max()

In [ ]:
# Check missing time steps in test.csv
get_missing_steps(df_test,'2011-01-20','2011-01-31','H')

In [ ]:
# Let's append a new row with max predicted date
obj = {}
for col in df.columns:
    obj[col] = [np.nan]
df = df.append(pd.DataFrame(obj,index=[dt_predict_max]))

In [ ]:
# Resample entire data at 1 hour frequency
df = df.resample('1h').mean()

In [ ]:
# dataset has these features set to NaN
df.tail()

In [ ]:
# test file has features available for some of the time steps
df_test.tail()

In [ ]:
# Let's update data from test
df.update(df_test)

In [ ]:
# Now we have data for dynamic features
df.tail()

In [ ]:
# Every feature has NaN (missing values)
# We need to fill in these missing features
df.isna().any()

In [ ]:
# Let's look at some missing steps
# Jan 2011
get_missing_steps(df_test,'2011-01-20','2011-01-31','H')

In [ ]:
# Dec 2012
get_missing_steps(df_test,'2012-12-20','2012-12-31','H')

In [ ]:
df['2011-01-26']

In [ ]:
df['2012-12-24']

In [ ]:
df['season']['2011-01'].plot()

In [ ]:
df['season']['2012-12-17':'2012-12-25'].plot()

In [ ]:
# https://stackoverflow.com/questions/19966018/pandas-filling-missing-values-by-mean-in-each-group
# For these ['holiday','workingday','season'] columns, we simply need to look for another hour in the same day
# for which value is available and use it to fillna
# Holiday won't change hour to hour
# Holiday,Workingday and Season are the same for all hours of the day
group_ymd = df[['holiday','workingday','season']].groupby([df.index.year,df.index.month,df.index.day])

for col in ['holiday','workingday','season']:
    print(col)
    df[col] = group_ymd[col].transform(lambda x: x.fillna(x.max()))

In [ ]:
df['season']['2011-01'].plot()

In [ ]:
df['season']['2012-12-17':'2012-12-25'].plot()

In [ ]:
df['season'].plot()

In [ ]:
# Remaining NaN values in features, simply ffill
for col in ['weather', 'temp', 'atemp', 'humidity', 'windspeed']:
    df[col] = df[col].fillna(method='ffill')

In [ ]:
df['2011-01-26']

In [ ]:
df['2012-12-24']

In [ ]:
# Now, only the rental count columns [count,registered,casual] should have NaN to indicate missing values
df.isna().any()

In [ ]:
df_dynamic_feat = df[dynamic_features]

In [ ]:
df_dynamic_feat.head()

In [ ]:
df_dynamic_feat.tail()

In [ ]:
# Missing target values that we need to predict and fill
df['2012-01':'2012-02']['count'].plot()

In [ ]:
time_series_test = []
time_series_training = []

for t in target_values:
    time_series_test.append(df[dt_test_range[0]:dt_test_range[1]][t])
    time_series_training.append(df[dt_train_range[0]:dt_train_range[1]][t])

In [ ]:
# Dynamic features are the same for count, registered, casual
dynamic_features_test = df_dynamic_feat [dt_test_range[0]:dt_test_range[1]]
dynamic_features_training = df_dynamic_feat[dt_train_range[0]:dt_train_range[1]]

In [ ]:
time_series_test[0].plot(label='test')
time_series_training[0].plot(label='train')
plt.legend()
plt.show()

In [ ]:
def encode_target(ts):
    return [x if np.isfinite(x) else "NaN" for x in ts]  

def encode_dynamic_feat(dynamic_feat):  
    l = []
    for col in dynamic_feat:
        assert (not dynamic_feat[col].isna().any()), col  + ' has NaN'             
        l.append(dynamic_feat[col].tolist())
    return l

def series_to_obj(ts, cat=None, dynamic_feat=None):
    obj = {"start": str(ts.index[0]), "target": encode_target(ts)}
    if cat is not None:
        obj["cat"] = cat
    if dynamic_feat is not None:
        obj["dynamic_feat"] = encode_dynamic_feat(dynamic_feat)
    return obj

def series_to_jsonline(ts, cat=None, dynamic_feat=None):
    return json.dumps(series_to_obj(ts, cat, dynamic_feat))   

In [ ]:
series_to_jsonline(time_series_training[0][:5], dynamic_feat=dynamic_features_training[:5])

In [ ]:
encoding = "utf-8"
with open("train_dynamic_feat.json", 'wb') as fp:
    for ts in time_series_training:
        fp.write(series_to_jsonline(ts,dynamic_feat=dynamic_features_training).encode(encoding))
        fp.write('\n'.encode(encoding))

In [ ]:
with open("test_dynamic_feat.json", 'wb') as fp:
    for ts in time_series_test:
        fp.write(series_to_jsonline(ts,dynamic_feat=dynamic_features_test).encode(encoding))
        fp.write('\n'.encode(encoding))

In [ ]:
df.to_csv('all_data_dynamic_feat.csv',index=True,index_label='datetime')

In [ ]:
for ts in time_series_test:
    print (len(ts),ts.name)

In [ ]:
for ts in time_series_training:
    print (len(ts),ts.name)